In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [91]:
df = pd.read_csv('../raw_data/test.csv').drop(columns=['id'])
df.columns
tmp= [df.columns[1]] + [df.columns[0]] +list(df.columns[2:])
df = df[tmp]
data = np.array(df)
df.iloc[:3]

,mv_fluency,votes,AvgCharPerSent,AvgSpeed,CharNumVar,CharNum_25Per,CharNum_50Per,CharNum_75Per,CharNum_Avg,CharNum_Max,...,TotalAdvNum,TotalCharNum,TotalNounNum,TotalPauseWordNum,TotalQuestionSentNum,TotalSentNum,TotalVerbNum,TotalXNum,VoiceLen,VoiceOverFilePercent
0,1,5,19.666667,2.478927,36.222222,15.5,17.0,22.50,19.666667,28,...,2,59,3,1,0,3,6,2,23.386,0.928248
1,1,5,17.833333,2.914322,207.472222,8.0,14.5,21.75,17.833333,46,...,9,107,5,9,0,6,17,1,35.490,0.879416
2,0,0,5.966667,1.949080,41.232222,2.0,5.0,6.75,5.966667,35,...,4,179,6,2,3,30,43,22,86.530,0.581363


In [92]:
#col1:label, col2:weight

In [94]:
data = inferWeight(data)

In [74]:
def inferWeight(data, alpha=None, beta=None):    
    votes = data[:,1]
    maxVote = max(votes)
    weights = []
    for i in range(votes.shape[0]):
        v = votes[i]
        if(v>=(1+maxVote)/2):
            if(alpha==None or beta==None):
                weights.append(float(v/maxVote))
            else:
                weights.append(float((v+alpha)/(maxVote+alpha+beta)))
        else:
            if(alpha==None or beta==None):
                weights.append(1-float(v/maxVote))
            else:
                weights.append(float((maxVote-v+alpha)/(maxVote+alpha+beta)))
    data[:,1] = weights
    return data

def splitFeatureWeight(x):
    return x[:,1:], x[:,0]

def createGroupsRandom(data, groupSize=int(1e5)):
    positive = data[np.where(data[:,0]==1)]
    negative = data[np.where(data[:,0]==0)]
    posNum = positive.shape[0]
    negNum = negative.shape[0]
    idx = np.random.randint(low=0, high=posNum, size=groupSize)
    query = np.array([positive[i,1:] for i in idx])
    posDoc = shuffle(query)
    idx = np.random.randint(low=0, high=negNum, size=groupSize)
    negDoc0 = np.array([negative[i,1:] for i in idx])
    negDoc1 = shuffle(negDoc0)
    negDoc2 = shuffle(negDoc0)
    
    query, _ = splitFeatureWeight(query)
    posDoc, posDocW = splitFeatureWeight(posDoc)
    negDoc0, negDoc0W = splitFeatureWeight(negDoc0)
    negDoc1, negDoc1W = splitFeatureWeight(negDoc1)
    negDoc2, negDoc2W = splitFeatureWeight(negDoc2)
    
    groups = (query, posDoc, negDoc0, negDoc1, negDoc2)
    weights = (posDocW, negDoc0W, negDoc1W, negDoc2W)
    return groups, weights

In [95]:
groups, weights = createGroupsRandom(data)

In [96]:
groups[0].shape

(100000, 50)

In [97]:
weights[0].shape

(100000,)

In [98]:
weights[0]

array([1. , 0.2, 1. , ..., 0.4, 0.2, 0.4])